# PyTorch

I have tried to make the flow of this notebook very similar to the previous notebook on Tensorflow, so that you can follow the similar logic behind the processes, as well as see a few differences.

[This is based on work shown here: https://towardsdatascience.com/your-first-neural-network-in-pytorch-725631ae0fc]

# Import the libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Get the data

In [ ]:
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv'
iris = pd.read_csv(path, header=None)

In [ ]:
iris

In [ ]:
iris.columns = ['sepalLength','sepalWidth','petalLength','petalWidth','species']

In [ ]:
mappings = {
   'Iris-setosa': 0,
   'Iris-versicolor': 1,
   'Iris-virginica': 2
}
iris['species'] = iris['species'].apply(lambda x: mappings[x])

In [ ]:
iris

In [ ]:
X = iris.drop('species', axis=1).values
y = iris['species'].values

# Use 2/3 of the data for training, and reserve 1/3 for testing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Convert the variables into tensor format for PyTorch manipulations

In [ ]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

# Define the model

* The init method specifies the layers, with the number of incoming and outgoing neural network connections
* The forward method specifies the activation functions
* (a dimension of -1 is just Python-speak for the last dimension)

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Linear(in_features=4, out_features=16)
        self.hidden1 = nn.Linear(in_features=16, out_features=8)
        self.output = nn.Linear(in_features=8, out_features=3)
 
    def forward(self, x):
        x = F.relu(self.input(x))
        x = F.relu(self.hidden1(x))
        x = F.softmax(self.output(x),dim=-1)
        return x

In [ ]:
model = MLP()
model

# Define the methods for assessing error levels during training

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train the model

In [ ]:
epochs = 150
loss_arr = []
for i in range(epochs):
   y_hat = model.forward(X_train)
   loss = criterion(y_hat, y_train)
   loss_arr.append(loss)
 
   if i % 10 == 0:
       print(f'Epoch: {i} Loss: {loss}')
 
   optimizer.zero_grad()
   loss.backward()
   optimizer.step()

# Evaluate the model

In [ ]:
preds = []
with torch.no_grad():
   for val in X_test:
       y_hat = model.forward(val)
       preds.append(y_hat.argmax().item())

In [ ]:
df = pd.DataFrame({'Y': y_test, 'YHat': preds})
df['Correct'] = [1 if corr == pred else 0 for corr, pred in zip(df['Y'], df['YHat'])]

In [ ]:
df.head()

In [ ]:
df['Correct'].sum() / len(df)

# Look at the evolution of Loss during training

In [ ]:
plt.plot(loss_arr)

# Using PyTorch (and Tensorflow) to analyze the Iris data set is probably significant overkill, but it gives you a flavor for the basic flow